In [4]:
from numpy import *
import operator
import pandas as pd
import random

In [6]:
data = pd.read_csv('../svm_data.csv')

# Define a Function to Select Random Sample Pairs Given the First Sample

In [19]:
# Randomly choose another sample
def select_random(i, m):
    return random.choice([j for j in list(range(m)) if j != i])

In [43]:
# Cut off alpha that is too big/small
def clip_alpha(alpha, high, low):
    if high < low:
        print('High bar is < low bar')
        return alpha
    if alpha > high:
        alpha = high
    if low > alpha:
        alpha = low
    return alpha

# Train SVM with SOM Method

In [40]:
def som_svm(data, C, thresh, max_iter):
    #split data
    train_data = mat(data[[column for column in data.columns if column != 'label']])
    labels = mat(data['label']).T
    b=0
    num_rows, num_cols = train_data.shape
    #initialization of alphas
    alphas = mat(zeros((num_rows,1)))
    
    iter = 0 #iterations = how many times go through each sample without updating 
    while (iter < max_iter):
        
        alpha_pairs_changed=0
        for i in range(num_rows):
            #calculate current x -- alpha * xxt + b
            fxi = float(multiply(alphas,labels).T*(train_data*train_data[i,:].T)) + b
            #calculate error
            ei = fxi - float(labels[i])
            
            #if error > thresh
            if ((labels[i]*ei < -thresh) and (alphas[i] < C)) or ((labels[i]*ei > thresh) and (alphas[i] > 0)):
                j = select_random(i, num_rows)#randomly choose another sample to pair with i
                fxj = float(multiply(alphas,labels).T*(train_data*train_data[j,:].T)) + b
                ej = fxj - float(labels[j])
                
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                if (labels[i] != labels[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H: 
                    print("L==H")
                    continue
                   
                #calculate eta
                eta = 2.0 * train_data[i,:]*train_data[j,:].T -train_data[i,:]*train_data[i,:].T -train_data[j,:]*train_data[j,:].T
                if eta >= 0: 
                    print("eta>=0")
                    continue
                    
                print(i, alphas[i], alphas[j], ei, ej, eta)
                #renew alpha j
                alphas[j] -= labels[j]*(ei - ej)/eta
                alphas[j] = clip_alpha(alphas[j],H,L)
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough")
                    continue
                    
                #calculate other parameters
                alphas[i] += labels[j]*labels[i]*(alphaJold - alphas[j])
                b1 = b - ei- labels[i]*(alphas[i]-alphaIold)*train_data[i,:]*train_data[i,:].T -labels[j]*(alphas[j]-alphaJold)*train_data[i,:]*train_data[j,:].T
                b2 = b - ej- labels[i]*(alphas[i]-alphaIold)*train_data[i,:]*train_data[j,:].T -labels[j]*(alphas[j]-alphaJold)*train_data[j,:]*train_data[j,:].T
                print(i, alphas[i], alphas[j], ei, ej, eta)
                
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else: 
                    b = (b1 + b2)/2.0
                alpha_pairs_changed += 1
                print("iter: %d i:%d, pairs changed %d" % (iter,i,alpha_pairs_changed))
                
        if (alpha_pairs_changed == 0): iter += 1
        else: iter = 0
        print("iteration number: %d" % iter, b)
    return b,alphas


# Example

In [46]:
b,alphas = som_svm(data,0.6,0.001,20)
print(b)
print(alphas)

L==H
L==H
L==H
L==H
L==H
L==H
6 [[ 0.]] [[ 0.]] -1.0 1.0 [[-58.33161935]]
6 [[ 0.03428672]] [[ 0.03428672]] -1.0 1.0 [[-58.33161935]]
iter: 0 i:6, pairs changed 1
7 [[ 0.]] [[ 0.]] [[ 0.29054225]] [[-1.1117834]] [[-29.7656959]]
7 [[ 0.04711214]] [[ 0.04711214]] [[ 0.29054225]] [[-1.1117834]] [[-29.7656959]]
iter: 0 i:7, pairs changed 2
L==H
10 [[ 0.]] [[ 0.]] [[ 0.33529243]] [[ 0.60038849]] [[-27.99431894]]
j not moving enough
11 [[ 0.]] [[ 0.]] [[ 0.02859355]] [[ 0.01666002]] [[-23.69019584]]
11 [[ 0.00050373]] [[ 0.00050373]] [[ 0.02859355]] [[ 0.01666002]] [[-23.69019584]]
iter: 0 i:11, pairs changed 3
L==H
23 [[ 0.]] [[ 0.]] [[ 0.44285002]] [[ 0.438918]] [[-26.68751861]]
23 [[ 0.00014734]] [[ 0.00014734]] [[ 0.44285002]] [[ 0.438918]] [[-26.68751861]]
iter: 0 i:23, pairs changed 4
25 [[ 0.]] [[ 0.00050373]] [[-0.26280842]] [[-0.44003357]] [[-0.21521253]]
j not moving enough
L==H
L==H
31 [[ 0.]] [[ 0.]] [[-0.11350104]] [[-0.57693143]] [[-38.50300896]]
j not moving enough
34 [[ 0.]] 